此示例显示可以组合因子以创建新因子，可以使用`mask`参数对因子进行排名，以及如何使用`pipeline`API创建多头和空头列表。

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-1-1 --end 2018-4-25
from logbook import Logger
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

log = Logger('回测')

def initialize(context):

    sma_short = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)
    sma_long = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=100)

    # Combined factors to create new factors
    sma_quotient = sma_short / sma_long

    # Create a screen to remove penny stocks
    remove_penny_stocks = sma_short > 1.0

    sma_rank = sma_quotient.rank(mask=remove_penny_stocks)

    # Rank a factor using a mask to ignore the values we're
    # filtering out by passing mask=remove_penny_stocks to rank.
    longs = sma_rank.top(200, mask=remove_penny_stocks)
    shorts = sma_rank.bottom(200, mask=remove_penny_stocks)

    pipe_columns = {'longs': longs, 'shorts': shorts, 'sma_rank': sma_rank}

    # Use multiple screens to narrow the universe
    pipe_screen = (longs | shorts)

    pipe = Pipeline(columns=pipe_columns, screen=pipe_screen)
    attach_pipeline(pipe, 'example')


def before_trading_start(context, data):
    context.output = pipeline_output('example')

    context.longs = context.output[context.output.longs]
    context.shorts = context.output[context.output.shorts]

    context.security_list = context.shorts.index.union(context.longs.index)

    # Print the 5 securities with the lowest sma_rank.
    print("SHORT LIST")
    log.info(
        "\n" + str(context.shorts.sort_values(['sma_rank'], ascending=True).head()))

    # Print the 5 securities with the highest sma_rank.
    print("LONG LIST")
    log.info(
        "\n" + str(context.longs.sort_values(['sma_rank'], ascending=False).head()))

SHORT LIST
[2018-04-29 19:20:53.955240] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
思美传媒(002712)  False    True       3.0
天润曲轴(002283)  False    True       4.0
金石东方(300434)  False    True       5.0
LONG LIST
[2018-04-29 19:20:53.961474] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3026.0
上海凤凰(600679)   True   False    3025.0
齐翔腾达(002408)   True   False    3024.0
卓郎智能(600545)   True   False    3023.0
南洋科技(002389)   True   False    3022.0
SHORT LIST
[2018-04-29 19:20:53.975191] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
天润曲轴(002283)  False    True       3.0
金石东方(300434)  False    True       4.0
思美传媒(002712)  False    True       5.0
LONG LIST
[2018-04-29 19:20:53.982643] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3028.0
上海凤凰(600679)   True   False    3027.0


SHORT LIST
[2018-04-29 19:20:54.857150] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
LONG LIST
[2018-04-29 19:20:54.863060] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3069.0
江苏国信(002608)   True   False    3068.0
中航沈飞(600760)   True   False    3067.0
贝瑞基因(000710)   True   False    3066.0
柘中股份(002346)   True   False    3065.0
SHORT LIST
[2018-04-29 19:20:54.871890] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
LONG LIST
[2018-04-29 19:20:54.877115] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3071.0
江苏国信(002608)   True   False    3070.0


SHORT LIST
[2018-04-29 19:20:55.094040] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
科大国创(300520)  False    True       3.0
世名科技(300522)  False    True       4.0
微光股份(002801)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.098598] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3108.0
江苏国信(002608)   True   False    3107.0
ST云维(600725)   True   False    3106.0
贝瑞基因(000710)   True   False    3105.0
柘中股份(002346)   True   False    3104.0
SHORT LIST
[2018-04-29 19:20:55.108755] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
科大国创(300520)  False    True       3.0
世名科技(300522)  False    True       4.0
微光股份(002801)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.113198] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3109.0
江苏国信(002608)   True   False    3108.0


SHORT LIST
[2018-04-29 19:20:55.345190] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
世名科技(300522)  False    True       3.0
中科创达(300496)  False    True       4.0
华媒控股(000607)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.350065] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3127.0
江苏国信(002608)   True   False    3126.0
天山股份(000877)   True   False    3125.0
湖南天雁(600698)   True   False    3124.0
同力水泥(000885)   True   False    3123.0
SHORT LIST
[2018-04-29 19:20:55.360930] INFO: 回测: 
              longs  shorts  sma_rank
金冠电气(300510)  False    True       1.0
东方网络(002175)  False    True       2.0
世名科技(300522)  False    True       3.0
华媒控股(000607)  False    True       4.0
中科创达(300496)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.366595] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3130.0
江苏国信(002608)   True   False    3129.0


SHORT LIST
[2018-04-29 19:20:55.622516] INFO: 回测: 
               longs  shorts  sma_rank
金冠电气(300510)   False    True       1.0
四通股份(603838)   False    True       2.0
华媒控股(000607)   False    True       3.0
开尔新材(300234)   False    True       4.0
*ST河化(000953)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.628845] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3161.0
天山股份(000877)   True   False    3160.0
至纯科技(603690)   True   False    3159.0
江苏国信(002608)   True   False    3158.0
江阴银行(002807)   True   False    3157.0
SHORT LIST
[2018-04-29 19:20:55.637301] INFO: 回测: 
               longs  shorts  sma_rank
金冠电气(300510)   False    True       1.0
四通股份(603838)   False    True       2.0
华媒控股(000607)   False    True       3.0
开尔新材(300234)   False    True       4.0
*ST河化(000953)  False    True       5.0
LONG LIST
[2018-04-29 19:20:55.646296] INFO: 回测: 
              longs  shorts  sma_rank
中航沈飞(600760)   True   False    3164.0
天山股份(000877)   True   Fals

...

SHORT LIST
[2018-04-29 19:21:00.288886] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
宣亚国际(300612)   False    True       2.0
赛摩电气(300466)   False    True       3.0
ST云维(600725)   False    True       4.0
*ST众和(002070)  False    True       5.0
LONG LIST
[2018-04-29 19:21:00.295914] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3480.0
重庆钢铁(601005)   True   False    3479.0
贵州燃气(600903)   True   False    3478.0
泰禾集团(000732)   True   False    3477.0
新城控股(601155)   True   False    3476.0
SHORT LIST
[2018-04-29 19:21:00.305219] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
宣亚国际(300612)   False    True       2.0
赛摩电气(300466)   False    True       3.0
ST云维(600725)   False    True       4.0
*ST信通(600289)  False    True       5.0
LONG LIST
[2018-04-29 19:21:00.311227] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3481.0
重庆钢铁(601005)   True   Fals

SHORT LIST
[2018-04-29 19:21:00.568183] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
ST保千里(600074)  False    True       2.0
*ST信通(600289)  False    True       3.0
乐视网(300104)    False    True       4.0
顺威股份(002676)   False    True       5.0
LONG LIST
[2018-04-29 19:21:00.572543] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3493.0
泰禾集团(000732)   True   False    3492.0
重庆钢铁(601005)   True   False    3491.0
三六零(601360)    True   False    3490.0
华森制药(002907)   True   False    3489.0
SHORT LIST
[2018-04-29 19:21:00.583549] INFO: 回测: 
               longs  shorts  sma_rank
大连电瓷(002606)   False    True       1.0
ST保千里(600074)  False    True       2.0
乐视网(300104)    False    True       3.0
*ST信通(600289)  False    True       4.0
顺威股份(002676)   False    True       5.0
LONG LIST
[2018-04-29 19:21:00.589965] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3493.0
泰禾集团(000732)   True   Fals

SHORT LIST
[2018-04-29 19:21:00.847699] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
大连电瓷(002606)   False    True       3.0
*ST信通(600289)  False    True       4.0
ST巴士(002188)   False    True       5.0
LONG LIST
[2018-04-29 19:21:00.857100] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3494.0
重庆钢铁(601005)   True   False    3493.0
上海钢联(300226)   True   False    3492.0
用友网络(600588)   True   False    3491.0
华森制药(002907)   True   False    3490.0
SHORT LIST
[2018-04-29 19:21:00.871829] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
大连电瓷(002606)   False    True       3.0
*ST信通(600289)  False    True       4.0
ST巴士(002188)   False    True       5.0
LONG LIST
[2018-04-29 19:21:00.877676] INFO: 回测: 
              longs  shorts  sma_rank
贵州燃气(600903)   True   False    3495.0
重庆钢铁(601005)   True   Fals

SHORT LIST
[2018-04-29 19:21:01.135213] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
ST巴士(002188)   False    True       3.0
*ST信通(600289)  False    True       4.0
华仁药业(300110)   False    True       5.0
LONG LIST
[2018-04-29 19:21:01.139663] INFO: 回测: 
              longs  shorts  sma_rank
万兴科技(300624)   True   False    3503.0
贵州燃气(600903)   True   False    3502.0
合肥城建(002208)   True   False    3501.0
盘龙药业(002864)   True   False    3500.0
用友网络(600588)   True   False    3499.0
SHORT LIST
[2018-04-29 19:21:01.153667] INFO: 回测: 
               longs  shorts  sma_rank
乐视网(300104)    False    True       1.0
ST保千里(600074)  False    True       2.0
ST巴士(002188)   False    True       3.0
*ST信通(600289)  False    True       4.0
华仁药业(300110)   False    True       5.0
LONG LIST
[2018-04-29 19:21:01.158148] INFO: 回测: 
              longs  shorts  sma_rank
万兴科技(300624)   True   False    3503.0
合肥城建(002208)   True   Fals

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.009712,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.017593,0.021406,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.017435,0.083084,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011356,0.115404,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.016261,0.100950,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.014560,0.095760,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.007377,0.104382,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,0.002279,0.102578,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,0.002971,0.095975,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,0.002830,0.090519,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


**运行时间8.81秒**